In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES']="0"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH']="true"
#!pip install numpy==1.16.1
import numpy as np

In [2]:
from __future__ import print_function


import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,BatchNormalization
from keras.preprocessing.text import Tokenizer


max_words = 1000
print('Loading data...')

(x_train, y_train), (x_test, y_test) = reuters.load_data(path="reuters.npz",
                                                         num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Using TensorFlow backend.
/home/btek/.conda/envs/kerasgpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/btek/.conda/envs/kerasgpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/btek/.conda/envs/kerasgpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/btek/.conda/envs/kerasg

Loading data...
SEARCHING /home/btek/.keras/datasets/reuters.npz reuters.npz
46 classes
Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [3]:
# this is to allow focusing layer more room in the input domain
pad = 9
x_train_2 = np.zeros(shape=(x_train.shape[0],x_train.shape[1]+2*pad), dtype=x_train.dtype)
x_test_2 = np.zeros(shape=(x_test.shape[0],x_test.shape[1]+2*pad), dtype=x_test.dtype)
x_train_2[:,pad:-pad]=x_train
x_test_2[:,pad:-pad]=x_test
max_words2 = max_words+2*pad

In [12]:
from keras_utils import SGDwithLR, RMSpropwithClip
lr_dict = {'all':5e-5,
           'focus-1/Sigma:0': 0.001,'focus-1/Mu:0': 0.001,'focus-1/Weights:0': 0.0001,
           'focus-2/Sigma:0': 0.001,'focus-2/Mu:0': 0.001,'focus-2/Weights:0': 0.0001,
           'dense_1/Weights:0':5e-5}

mom_dict = {'all':0.9}
    
decay_dict = {'all':0.9, 'focus-1/Sigma:0': 0.1,'focus-1/Mu:0':0.1,
              'focus-2/Sigma:0': 0.1,'focus-2/Mu:0': 0.1}
batch_size = 16
e_i = x_train.shape[0] // batch_size

decay_epochs =np.array([e_i*30], dtype='int64')
    

clip_dict = {'focus-1/Sigma:0':(0.05,1.0),'focus-1/Mu:0':(0.0,1.0),
             'focus-2/Sigma:0':(0.05,1.0),'focus-2/Mu:0':(0.0,1.0)}

In [13]:
from keras import models
from keras import layers
from Kfocusing import FocusedLayer1D
from keras.optimizers import RMSprop, SGD



def build_model(N=512, mod='dense', optimizer_s='SGDwithLR',inp_shape=None):
    
    model = models.Sequential()
    #model.add(Dropout(0.02))
    if mod=='dense':
        model.add(Dense(N, activation='relu',name='dense-1',input_shape=inp_shape))
        model.add(Dropout(0.2))
        model.add(Dense(N, activation='relu',name='dense-2'))
    elif mod=='focused':
        model.add(FocusedLayer1D(units=N,
                                  name='focus-1',
                                  activation='linear',
                                  init_sigma=0.25, 
                                  init_mu='spread',
                                  init_w= None,
                                  train_sigma=True, 
                                  train_weights=True,
                                  train_mu = True,normed=2,input_shape=inp_shape))
        
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(0.2))
        model.add(FocusedLayer1D(units=N,
                                  name='focus-2',
                                  activation='linear',
                                  init_sigma=0.25, 
                                  init_mu='spread',
                                  init_w= None,
                                  train_sigma=True, 
                                  train_weights=True,
                                  train_mu = True,normed=2))
        
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, name='dense-3'))
    model.add(Activation('softmax'))
    

    if optimizer_s == 'SGDwithLR':
        opt = SGDwithLR(lr_dict, mom_dict,decay_dict,clip_dict, decay_epochs=decay_epochs)
    elif optimizer_s=='RMSpropwithClip':
        opt = RMSpropwithClip(lr=0.001, rho=0.9, epsilon=None, decay=None,clips=clip_dict)
    else:
        opt= SGD(lr=0.01, momentum=0.9)#, decay=None)
    
    
    
    model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [14]:
batch_size = 16
epochs = 15
model1 = build_model(N=150, mod='focused',optimizer_s=SGDwithLR,inp_shape=(max_words,))
history_focused = model1.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_data=(x_test,y_test))
score = model1.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

{'name': 'focus-1', 'input_shape': (1000,)}
{'name': 'focus-2'}
Train on 8982 samples, validate on 2246 samples
Epoch 1/15
 - 4s - loss: 2.0822 - acc: 0.4840 - val_loss: 2.6453 - val_acc: 0.3620
Epoch 2/15
 - 3s - loss: 2.4218 - acc: 0.3488 - val_loss: 2.4240 - val_acc: 0.3620
Epoch 3/15
 - 3s - loss: 2.4179 - acc: 0.3511 - val_loss: 2.4158 - val_acc: 0.3620
Epoch 4/15
 - 3s - loss: 2.4145 - acc: 0.3500 - val_loss: 2.4188 - val_acc: 0.3620
Epoch 5/15
 - 3s - loss: 2.4149 - acc: 0.3504 - val_loss: 2.4168 - val_acc: 0.3620
Epoch 6/15
 - 3s - loss: 2.4141 - acc: 0.3516 - val_loss: 2.4228 - val_acc: 0.3620
Epoch 7/15
 - 3s - loss: 2.4110 - acc: 0.3519 - val_loss: 2.4159 - val_acc: 0.3620
Epoch 8/15
 - 3s - loss: 2.4108 - acc: 0.3517 - val_loss: 2.4178 - val_acc: 0.3620
Epoch 9/15
 - 3s - loss: 2.4107 - acc: 0.3516 - val_loss: 2.4201 - val_acc: 0.3620
Epoch 10/15
 - 3s - loss: 2.4118 - acc: 0.3517 - val_loss: 2.4173 - val_acc: 0.3620
Epoch 11/15
 - 3s - loss: 2.4125 - acc: 0.3510 - val_loss

In [ ]:
batch_size = 16
epochs = 15
model2 = build_model(N=150, mod='dense',optimizer_s=RMSpropwithClip,inp_shape=(max_words,))
history_dense = model2.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)
score = model2.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])